<a href="https://colab.research.google.com/github/mehmetalianil/epoch-15/blob/master/Second_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First day

This day, we are going to do the following: 
* If not done yet, we need to sort out some prerequisites.
* We are going to generate a dataset of hand gestures, together.
* We might spend some time to gather and preprocess the data. (Never underestimate time spent for data-prep)
* Then we will start a Tensorflow session, and try to solve a classification problem of classifying these gestures correctly.

## Resources
* Arduino IDE download link https://www.arduino.cc/en/Main/Software
* Linux CH340 drivers patching guide https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all#linux
* Upload point for generated data: https://drive.google.com/drive/folders/138hSEWZyePWepjaOIrhxuYEvBwqLl2PM?usp=sharing
* A VERY helpful Colab written by François Chollet, developer of KERAS https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO
* Some numpy/pandas commands that might be useful https://elitedatascience.com/python-cheat-sheet
* We will be working with numpy arrays in high dimension. Here is how to slice them. https://www.pythoninformer.com/python-libraries/numpy/index-and-slice/

## Introduction

*There might be new faces in this session, therefore I would like to write an introduction.*

I am Mehmet Ali Anil, co-founder of Grus in High Tech Campus. I am a electrical engineer and physicist with particular interest on biologically inspired systems for a while now. Here are the places that you can contact me and read things I write: 

* Github https://github.com/mehmetalianil
* StackOverflow https://stackexchange.com/users/462542/mehmet-ali-anil
* LinkedIn https://www.linkedin.com/in/mehmetalianil/
* Our blog https://blog.grusbv.com/
* Twitter https://twitter.com/maanil_ee 
* Podcast {{Work In Progress}}

## Why microcontrollers? Isn't ML the job for, like endless server farms?
It doesn't have to be. It might have been a bad idea before, but the unexpected success of deep learning on specific problems has laid the groundwork for its widespread applicability. And systems with small resources, given a small problem to solve excel in applications that require: 

* Autonomy
* Privacy
* Speed (as in latency)
* Energy efficiency

In short, life is what is here and now, and why not solve problems where they are?

There are many ways to incorporate machine learning in embedded devices, but I am personally intrigued by the idea of using microcontrollers that are essentially ubiquotus now. [This Quora link](https://www.quora.com/How-many-embedded-systems-are-there) states that there might be 75-100 billions of devices housing microcontrollers. Due to their nature, these microcontrollers are used for tasks that involve deterministic control. They are so cheap and widespread that they might just as well be hardware neurons in our designs. We just need to learn how to program them. That is what we are up to. 

## We have some prerequisites.

### Get a serial monitor

We need a serial terminal to get our data out of our sparkfun boards. There are a few candidates:

* Docklight
* Tera Term
* screen (Linux terminal)
* Arduino IDE (I tested it here)

I have chosen Arduino because it also has a plotter that graphs the serial data stream if it has comma separated values. 

### If using Linux, download patched CH340 drivers
The current driver in the Linux kernel needs to be patched before programming the device with high speeds around 1MHz. It might be okay today, since we will only program if we have the time, but keep it in mind, follow the guidelines here:

https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all#linux

## Lets generate data

* Plug the USB cable to the computer.
* Plug the serial converter board on, the dark board.
* Plug the red Sparkfun Edge board on the serial converter board. They should both be chip side up. (The sparkfun board has a GRN written on the top pin,  which is **not** and indication that it must be connected to GND on the serial converter.)
* Your board might have some of its lights on, don't worry. 
* Fire up Arduino IDE, `Ctrl`+`Shift`+`L` or `Tools>>Serial Plotter` to fire up the serial plotter. 
* Press the button on your board marked as RST while watching the Serial Plotter. Sparkfun edge board will send a record of 2 seconds woorth of accelerometer readings.
* Experiment with the accelerometer if you are into that sort of things. Check whether gravity is still 1G. 
* Hold the boards secure in your hand (you can hold by the serial board) and come up with a hand gesture that you can do within a second, something bold and makes a distinct waveform on the serial plotter. Start and stop without jerking the device. Practice here. (Each person, one gesture.)
* Name this gesture, draw it up on the board to avoid different people choosing very similar gestures.
* Close the serial plotter, fire up the serial monitor with `Ctrl`+`Shift`+`M` or `Tools>>Serial Monitor`
* Now, record as many gestures as you can. Press the RST button on the device, and execute the gesture. Try to get the gesture into the middle of the 2 minute window. Try to have a single way of holding the boards, start and stop without motion. 
* If there are some spurious signal, it is due to the cable and the connections. check them out and continue. I will clean them out by hand.
* After done, copy paste the output into a file named for your gesture, with the extension .csv. It should be like: "fistbump.csv"









In [0]:
# Run this command in order to download or update the dataset that we are building up from scratch!
!rm -rf epoch-15/
!gsutil -m cp -r gs://epoch-15 ./

##Enabling GPU in Colab

We need to be sure that we are using TF 2.0 and we would like to use the GPU instance of Google.

For that, we select `Runtime >> Change runtime type >> HW accelerator >> GPU`

We also instantiate tensorboard here, because it is nicer to have it on the top.

In [0]:
# This uninstalls default tensorflow and tensorboad in a Colab and installs the latest nightly snapshots. 
#!pip uninstall -y tensorflow tensorflow-gpu tensorboard
#!pip install  -q tfa-nightly tf-nightly==2.1.0.dev20191029 tb-nightly==2.1.0a20191029 tf-nightly-gpu==2.1.0.dev20191029
#!pip uninstall -y tensorflow tensorflow-gpu tensorboard
#!pip install -q  grpcio==1.24.3

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import tensorboard as tb
print(tb.__version__)
import tensorflow as tf
print(tf.__version__)

In [0]:
# To check whether we are enjoying a GPU. Thanks G!
!nvidia-smi

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import tensorboard as tb
print(tb.__version__)
import tensorflow as tf
print(tf.__version__)
%load_ext tensorboard
%tensorboard --logdir logs/fit

#Data preparation

Here, I have prepared most that is required. We are going to scan large .csv files that we have generated and get them into shape that we can use. We have an uknown number of samples. Let's call them `Nsamples`. Each sample has 3 "channels", ax,ay and az, and each channel has 1024 samples. 

We are looking at a sample that has a shape of (1024,3). So our dataset will have a shape of (Nsample,1024,3).

In Tensorflow, we prepare the data like this, since TF has its own facilities in preparing the data for training. For example, TF cen get this large dataset and jumble it, alter it, divide it up and create batched out of it. 

Here this script scans our folder and creates a dataset. Data is a Numpy array named `array_data`, the corresponding labels are named `array_labels`. If you need the labels as ints, `array_labels_ints`. If you want them as vectors, `array_data_categorical`.

I have thrown in a random plot for convenience.

In [0]:
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical


PATH_DATA = "epoch-15"
SAMPLE_SIZE = 256
MAXIMUM_ACCELERATION=32000 #[mG]

array_data = np.empty([0,SAMPLE_SIZE,3])
array_label = np.empty([0])
for root, dirs, files in os.walk(PATH_DATA):
    for file in files:
        print("Reading {}".format(os.path.join(root,file)))
        with open(os.path.join(root,file)) as f:
          for sample_counter, chunk in enumerate(pandas.read_csv(f, sep=',',comment='#', error_bad_lines=False, chunksize=SAMPLE_SIZE+1)):
              trimmed_chunk = np.expand_dims(chunk.iloc[:-1],axis=0) # has dimensions (1024,3) should be (1,1024,3)
              array_data = np.append(array_data,trimmed_chunk,axis=0).astype(np.float32) # We omit the ax,ay,az row
              array_label = np.append(array_label,file.split('.')[0])

array_data=(array_data.astype(float)+MAXIMUM_ACCELERATION/2)/MAXIMUM_ACCELERATION  # normalize to (0,1)
GESTURE_LIST = list(set(array_label))  ## Warning, this won't preserve order. Might find a better (manual?) way.
GESTURE_LIST_INT = range(len(GESTURE_LIST))
array_label_ints = [GESTURE_LIST.index(label) for label in array_label]
array_label_categorical = to_categorical(array_label_ints).astype(np.float32)

## Some random graph
sample = random.randint(1, len(array_label))
fig = plt.figure(figsize=(10, 8))
plt.plot(np.arange(SAMPLE_SIZE),array_data[sample,:,0])
plt.plot(np.arange(SAMPLE_SIZE),array_data[sample,:,1])
plt.plot(np.arange(SAMPLE_SIZE),array_data[sample,:,2])
plt.xlabel("Samples")
plt.ylabel("Acceleration [mG]")
plt.title(array_label[sample]+" #"+str(sample))

TfLiteFlattened = []
for ctr,datapoint in enumerate(array_data[sample,:,0]):
  TfLiteFlattened.append(datapoint,)
  TfLiteFlattened.append(array_data[sample,ctr,1])
  TfLiteFlattened.append(array_data[sample,ctr,2])
print(array_label_categorical.dtype)

Here, we can spend a lot of time. This is where we do some data science. 
We need to come up with a model that successfully categorizes these gestures. It means, we are going to determine: 
* Individual layers (Conv1D, Dense, Flatten, etc)
* Activation Functions (ReLU, Softmax, etc.)
* Data modifications (noise, augmentation, cropping)
* Optimizer
* Error function and metrics

It cannot be too large, since we have a limited FLASH memory in out small board. 

Try to come up with a successful model running on the cloud (here!) and we might embed it into the our boards. I have given you a very simple one here.

Check the Tensorboard up in this page. It will update the latest results back from your training. 

Following challenges:
* How is your training accuracy and validation accuracy? Is there room for better?
* How does changing BATCH size and EPOCH count effect the training?
* How can we make our model robust?
* Is there a way to find out which gestures are mistaken for each other? 


In [0]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import datetime


log_dir="logs/fit/model1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
file_writer = tf.summary.create_file_writer(log_dir)
array_data = np.expand_dims(array_data,axis=3)
#array_label_categorical = np.expand_dims(array_label_categorical,axis=2)
(trainX, testX, trainY, testY) = train_test_split(array_data,array_label_categorical,test_size=0.25, random_state=42)
print("Shapes: trainX {} testX {} trainY {} testY {}".format(trainX.shape, testX.shape, trainY.shape, testY.shape))

# 
#  CHALLENGE come up with a better model!
#

## MAA Simplified
#model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(SAMPLE_SIZE, 3)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(len(GESTURE_LIST), activation='softmax')) # softmax is used, because we only expect one gesture to occur per input
model.summary()

#model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(trainX, trainY, epochs=1000, batch_size=100, validation_data=(testX, testY),callbacks=[tensorboard_callback])
## This works for batch size 1 but somehow not for larger batches. Can we check?


# Second Day

Now, we are going to convert our Tensorflow model into a TF Lite model, and get something that we can put into our devices.

We generate a representative dataset that the converter uses to optimize the weight in such a way that we lose the least accuracy. 

Here I use the test dataset, but if I had a valication dataset, that would have been a better choice, or I could have used the whole dataset for generalizability. 



In [0]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

## Let's print the 
print(np.expand_dims(testX[0].astype(np.float32),axis=0).shape)

def representative_dataset_generator():
  for value in testX:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.expand_dims(value.astype(np.float32),axis=0)]
converter.representative_dataset = representative_dataset_generator
## represantative dataset generator requires a lits of one item of shape (1,N,3)
tflite_model = converter.convert()
open("gesture_quantized.tflite", "wb").write(tflite_model)



import os
basic_model_size = os.path.getsize("gesture_model.tflite")
quantized_model_size = os.path.getsize("gesture_quantized.tflite")
print("Model is normally {} bytes, but {} bytes when quantized".format(basic_model_size,quantized_model_size))

Let's explore our quantized dataset. 
Check different layers and how they are interpreted. 

Check the graph, how much accuracy did we lose?

In [0]:
import pprint
# Instantiate an interpreter for each model
gesture_model = tf.lite.Interpreter('gesture_model.tflite')
gesture_model_quantized = tf.lite.Interpreter('gesture_quantized.tflite')

# Allocate memory for each model
gesture_model.allocate_tensors()
gesture_model_quantized.allocate_tensors()

# Get indexes of the input and output tensors
gesture_model_input_index = gesture_model.get_input_details()[0]["index"]
gesture_model_output_index = gesture_model.get_output_details()[0]["index"]
gesture_model_quantized_input_index = gesture_model_quantized.get_input_details()[0]["index"]
gesture_model_quantized_output_index = gesture_model_quantized.get_output_details()[0]["index"]

print(gesture_model.get_input_details())
print(gesture_model.get_output_details())
print(gesture_model_quantized.get_input_details())
print(gesture_model_quantized.get_output_details())
pprint.pprint(gesture_model.get_tensor_details())
pprint.pprint(gesture_model_quantized.get_tensor_details())

predictions = model.predict(testX)
# Create arrays to store the results
gesture_model_predictions = []
gesture_model_quantized_predictions = []

# Run each model's interpreter for each value and store the results in arrays
for x_value in testX:
  # Create a 2D tensor wrapping the current x value
  x_value_tensor = tf.convert_to_tensor([x_value], dtype=np.float32)
  # Write the value to the input tensor
  gesture_model.set_tensor(gesture_model_input_index, x_value_tensor)
  # Run inference
  gesture_model.invoke()
  # Read the prediction from the output tensor
  gesture_model_predictions.append(
      gesture_model.get_tensor(gesture_model_output_index)[0])
  # Do the same for the quantized model
  gesture_model_quantized.set_tensor(gesture_model_quantized_input_index, x_value_tensor)
  gesture_model_quantized.invoke()
  gesture_model_quantized_predictions.append(
      gesture_model_quantized.get_tensor(gesture_model_quantized_output_index)[0])

# See how they line up with the data
fig = plt.figure(figsize=(10, 8))
plt.clf()
plt.title('Comparison of various models against actual values')
plt.plot(range(len(testX)), [np.argmax(label) for label in testY] , 'bo', label='Actual')
plt.plot(range(len(testX)), [np.argmax(label) for label in predictions], 'ro', label='Original predictions')
plt.plot(range(len(testX)), [np.argmax(label) for label in gesture_model_predictions], 'bx', label='Lite predictions')
plt.plot(range(len(testX)), [np.argmax(label) for label in gesture_model_quantized_predictions], 'gx', label='Lite quantized predictions')
plt.legend()
plt.show()



Now, we use a tool called xxd in order to export our model out. 

In [0]:
!apt-get -qq install xxd

!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_quantized.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

!echo "const unsigned char model[] = {" > /content/model_quantized.h
!cat gesture_quantized.tflite | xxd -i      >> /content/model_quantized.h
!echo "};"                              >> /content/model_quantized.h

import os
model_h_size = os.path.getsize("model.h")
model_quantized_h_size = os.path.getsize("model_quantized.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print(f"Header file, model.h, is {model_quantized_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Here, you can actually clone the repository to you computer, and edit there. 

Here, for sanity, we are going to cat the files into the cell here, and you are going to copy it in another cell, and save it with %%writefile filename.py

In [0]:
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout v2.1.0-rc0
# the list of gestures that data is available for

In [0]:
%cd /content/tensorflow/lite/experimental/micro/examples/
!cat magic_wand_test.cc


In [0]:
%%writefile magic_wand_test.py

In [0]:
!cat magic_wand_mode_data.cc

In [0]:
%%writefile magic_wand_test.py

In [0]:
%cd /content/tensorflow
!make -f tensorflow/lite/experimental/micro/tools/make/Makefile TARGET=sparkfun_edge magic_wand_bin

This command is needed because there is a mistake in the naming of the file.

In [0]:
!cp tensorflow/lite/experimental/micro/tools/make/downloads/AmbiqSuite-Rel2.0.0/tools/apollo3_scripts/keys_info0.py \
tensorflow/lite/experimental/micro/tools/make/downloads/AmbiqSuite-Rel2.0.0/tools/apollo3_scripts/keys_info.py

This command is required to create a secure binary to flash into the board.

In [0]:
!pip install pycrypto pyserial

%run tensorflow/lite/experimental/micro/tools/make/downloads/AmbiqSuite-Rel2.0.0/tools/apollo3_scripts/create_cust_image_blob.py \
--bin tensorflow/lite/experimental/micro/tools/make/gen/sparkfun_edge_cortex-m4/bin/magic_wand.bin \
--load-address 0xC000 \
--magic-num 0xCB \
-o main_nonsecure_ota \
--version 0x0

If you want to help me, tahe a look at this issue report at TFlite googlegroups.


https://groups.google.com/a/tensorflow.org/forum/?utm_medium=email&utm_source=footer#!msg/tflite/PTcmPMOpJQ4/9IRYPpM3CwAJ